In [1]:
import numpy  as np
import glob 
import os 
import gc 
from   pathlib import Path
from   typing import  List, Tuple, Callable, Any 
from   dataclasses import dataclass, asdict, field
from   pprint import pprint
from   functools import reduce

import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px

import modin.pandas as pd 
import ray 
os.environ["MODIN_ENGINE"] = "ray"
ray.init(ignore_reinit_error=True)

import wandb
print(f'wandb version: {wandb.__version__}')
%env "WANDB_NOTEBOOK_NAME" "m5-ts-model"
wandb.login()

2021-05-27 15:29:53,587	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: akamlani (use `wandb login --relogin` to force relogin)
wandb version: 0.10.30
env: "WANDB_NOTEBOOK_NAME"="m5-ts-model"


True

In [2]:
from    utils.config import load_config
from    utils.models import ModelForecastDense, ModelForecastBiLSTM, ModelForecastLSTM, ModelForecastLSTMOneShot

import  flexassist.integrations.wandb.lifecycle       as lc 
import  flexassist.integrations.sklearn.encoder       as enc
import  flexassist.integrations.tensorflow.dataloader as dl 
import  flexassist.integrations.tensorflow.encoder    as tfenc
import  flexassist.integrations.tensorflow.modeler    as tfmdl
import  flexassist.integrations.tensorflow.trainer    as trainer
import  flexassist.core.system.writer                 as wr 

## Ingestion

In [8]:
np.random.seed(42)
config_files:List[str] = glob.glob('./config' + '/*')
config = load_config('./config')

In [9]:
datasets_dir   = Path(config['experiment_parameters']['datasets_dir'])
transform_dir  = Path(datasets_dir)/'transformers'
df_cal         = pd.read_csv(Path(transform_dir)/'calendar.csv')    
df_prices      = pd.read_csv(Path(transform_dir)/'sell_prices.csv')    
df_train       = pd.read_csv(Path(transform_dir)/'train.csv') 
df_train.shape   

(30490, 1947)

## Transforms

In [10]:
# transform into just time series
ts_cols               = [col for col in df_train.columns if 'd_' in col]
lvl_cols              = [col for col in df_train.columns if 'd_' not in col]
df_train_daily_qty    = df_train[ts_cols].T
# configure time series
start_day             = config['architecture']['start_period']    # to avoid many days that have 
lookback_period       = config['architecture']['lookback_period'] # len(df_train_daily_qty) - start_day; to avoid those days with no transactions
time_steps            = config['architecture']['time_steps']      # how many lags to account for 
horizon               = config['architecture']['horizon']         # how many timesteps to project into future
output_width          = config['architecture']['output_width']    # for one week worth of ouput predictions
# calculate remaining days
df_train_daily_qty_lb = df_train_daily_qty[-lookback_period:]
num_days              = len(df_train_daily_qty_lb.index)
# calculate partitions
val_part_size         = horizon * 6
test_part_size        = horizon
df_test_part          = df_train_daily_qty_lb[-test_part_size:]                 # Days(1914 - 1941) ~ 1 month
df_valid_part         = df_train_daily_qty_lb[-val_part_size:-test_part_size]   # Days(1774 - 1913) ~ 6 months
df_train_part         = df_train_daily_qty_lb[0:-val_part_size]                 # Days(351  - 1773) ~ 2.3 years

In [11]:
# scale data
from sklearn.preprocessing import MinMaxScaler, StandardScaler
#scaler = StandardScaler()
scaler = MinMaxScaler(feature_range = (0,1))
df_train_part_sc = scaler.fit_transform(df_train_part)
df_valid_part_sc = scaler.transform(df_valid_part)
df_test_part_sc  = scaler.transform(df_test_part)
print(f"Partition Sizes: {df_train_part_sc.shape}, {df_valid_part_sc.shape}, {df_test_part_sc.shape}")
# get properties of normalize the the continuous columns
# scaler_props = zip([np.min, np.max], [scaler.data_min_, scaler.data_max_])
# scaler_props = zip([np.mean, np.var, np.std], [scaler.mean_, scaler.var_, np.sqrt(scaler.var_)])
# scaler_props = {k.__name__: v[-1].round(4) for k,v in scaler_props}
# print(f"Scaler Properties: {scaler_props}")
#print(f"Training Data: mu:  {df_train_part_sc.mean().round(5)}, std: {df_train_part_sc.std().round(5)}")
print(f"Training Data: min: {df_train_part_sc.min().round(5)}, max: {df_train_part_sc.max().round(5)}")

Partition Sizes: (1423, 30490), (140, 30490), (28, 30490)
Training Data: min: 0.0, max: 1.0


In [12]:
dataprep_dir = Path(datasets_dir)/'dataprep'
wr.write_csv(pd.DataFrame(df_train_part_sc), dataprep_dir, 'training_normalized.csv')    
wr.write_csv(pd.DataFrame(df_valid_part_sc), dataprep_dir, 'validation_normalized.csv')    
wr.write_csv(pd.DataFrame(df_test_part_sc),  dataprep_dir, 'test_normalized.csv')    

'experiments/snapshots/m5-fcst-base/exports/artifacts/datasets/dataprep/test_normalized.csv'

## Time Series Models 

In [13]:
@dataclass 
class SearchQuery:
    model_cls:Any
    gen_name:str 
    gen_params:dict 
    model_name:str               = field(default='model_template') 
    tracker_task:lc.JobTrackInfo = None 

def generator(df:pd.DataFrame, features:List[int], num_timesteps:int, out_width:int, horizon:int) -> dict:
    "greate a generator based on input configuration"
    meta_config:dict = dict (
        num_timesteps = num_timesteps,
        features      = features, 
        num_features  = len(features),
        input_shape   = (num_timesteps, len(features)),
        label_width   = out_width,
        horizon       = horizon 
    )
    gen = dl.WindowDataLoader(input_width=num_timesteps, label_width=out_width, horizon=horizon)
    # data frame is based on training configuration
    gen.register_frame(df, features)
    gen.register_metadata(meta_config)
    return gen

def train_and_evaluate(gen, partition_data:tuple, model, model_name:str, configuration:dict, verbose:bool=False):
    """train and evaluate model on dataset generators

    Examples:
    >>> features = [19,24,48]
    >>> single_step_win = generator(df_train_part_sc, features, num_timesteps=1, out_width=1, horizon=1)
    >>> partitions = (df_train_part_sc, df_valid_part_sc, df_test_part_sc)
    >>> model, df  = train_and_evaluate(single_step_win, partitions, model=model_dense, model_name='dense', configuration=config)
    """
    # generators 
    batch_size = configuration['training_parameters']['batch_size']
    train, valid, test = partition_data
    train_gen = gen.make_dataset(train[:, gen.column_indices], shuffle_en=True,  batch_size=batch_size)
    valid_gen = gen.make_dataset(valid[:, gen.column_indices], shuffle_en=False, batch_size=batch_size)
    test_gen  = gen.make_dataset(test[:,  gen.column_indices], shuffle_en=False, batch_size=batch_size)
    if verbose:
        get_dataset_spec  = lambda dataset: dataset.element_spec
        print(get_dataset_spec(train_gen))
    # training 
    trainer_svc = trainer.Trainer(configuration)
    model, df_history = trainer_svc.train(model_name, model, train_gen, valid_gen, **{'cb_wandb':True})
    # evaluation (during training, we use validation data)
    scores    = model.evaluate(valid_gen, return_dict=True)
    df_scores = pd.DataFrame.from_dict(scores, orient='index', columns=['score']).round(3).T
    return model, df_history, df_scores 

def search(model_data:List[SearchQuery], df:pd.DataFrame, partition_data:tuple, feature_cols:List[int], configuration:dict):
    "iterates and searches over model data for training and scoring"
    df_scoring = pd.DataFrame()
    for query in model_data:
        gen_win = generator(pd.DataFrame(df), features=feature_cols, **query.gen_params)
        cfg     = gen_win.meta_config
        model   = query.model_cls (
            input_shape     = cfg['input_shape'], 
            num_timesteps   = cfg['num_timesteps'],
            output_width    = cfg['label_width'],
            num_features    = cfg['num_features'],
            horizon         = cfg['horizon']
        )
        # lifefycle runner, search over models 
        with lc.LifeCycleSvc.execute(query.tracker_task) as cxt: 
            model, df_history, df_scores = train_and_evaluate (
                gen             = gen_win, 
                partition_data  = partition_data, 
                model           = model, 
                model_name      = query.model_name, 
                configuration   = configuration
            )
            # scoring on the validation set
            df_scoring = df_scoring.append(df_scores)
            df_scoring = df_scoring.rename(index={'score':query.model_name})
    df_scoring = df_scoring.rename_axis('scores')
    return df_scoring

In [23]:
### Define Lifecycle Context Runner Defaults 
# Define init configuration lifecycle context for each job 
project_config  = config['project']
tracker = lc.JobTrackInfo(**dict(
    name    = '{}:{}',
    project = project_config['name'], 
    entity  = project_config['entity'],
    group   = ":".join(['exp', 'modeltype', 'timeseries']),
    # default main configuration used which will be added to 
    config  = config, 
    tags    = project_config['tags'] + ['model experimentation'],
    notes   = "service model experimentation execution"
))
tracker.format_job_type(cycle=lc.JobTrackingCycle.Experiment, action=lc.ActionType.Execution)
# selecting top 25 of items to train models on
features    = pd.read_csv(Path(transform_dir)/'top_id_indices.csv')['indice'].values.tolist()   
partitions  = (df_train_part_sc, df_valid_part_sc, df_test_part_sc)

In [24]:
# define configurations to be used
single_in_step            = dict(num_timesteps=1,           out_width=1,             horizon=1)
wide_in_step              = dict(num_timesteps=time_steps,  out_width=1,             horizon=1)
wide_in_out               = dict(num_timesteps=time_steps,  out_width=output_width,  horizon=1)
wide_in_out_oneshot_hz_wk = dict(num_timesteps=time_steps,  out_width=output_width,  horizon=horizon//7)
wide_in_out_oneshot_hz    = dict(num_timesteps=time_steps,  out_width=output_width,  horizon=horizon)
# define models to be used 
database = [
    SearchQuery(model_cls=ModelForecastDense,       gen_name='single_in_step',               gen_params=single_in_step), 
    SearchQuery(model_cls=ModelForecastDense,       gen_name='wide_in_step',                 gen_params=wide_in_step), 
    SearchQuery(model_cls=ModelForecastLSTM,        gen_name='wide_in_step',                 gen_params=wide_in_step), 
    SearchQuery(model_cls=ModelForecastBiLSTM,      gen_name='wide_in_step',                 gen_params=wide_in_step), 
    SearchQuery(model_cls=ModelForecastLSTMOneShot, gen_name='wide_in_out',                  gen_params=wide_in_out), 
    SearchQuery(model_cls=ModelForecastLSTMOneShot, gen_name='wide_in_out_oneshot_hz_wk',    gen_params=wide_in_out_oneshot_hz_wk), 
    SearchQuery(model_cls=ModelForecastLSTMOneShot, gen_name='wide_in_out_oneshot_hz',       gen_params=wide_in_out_oneshot_hz) 
]
# configure tracker object for each entry 
for entry in database: 
    job_dict             = asdict(tracker)
    tracker_job          = lc.JobTrackInfo(**job_dict)
    # reuse and configure from main tracker configured
    tracker_job.group    = tracker.group 
    tracker_job.name     = tracker.name.format(entry.model_cls.__name__.lower(), entry.gen_name)
    tracker_job.job_type = tracker.job_type
    tracker_job.config.update({'generator':entry.gen_params})
    # add a tracking parameter 
    entry.tracker_task   = tracker_job 
    entry.model_name     = ":".join([entry.model_cls.__name__.lower(), entry.gen_name])
# perform search with context of spawning jobs
df_scores_eval = search(database, df_train_part_sc, partitions, features, config)
df_scores_eval

wandb: WARNING When using several event log directories, please call wandb.tensorboard.patch(root_logdir="...") before wandb.init
Epoch 1/30
45/45 [==============================] - 1s 8ms/step - loss: 0.0298 - mean_absolute_error: 0.1276 - mean_squared_error: 0.0298 - root_mean_squared_error: 0.1727 - val_loss: 0.0111 - val_mean_absolute_error: 0.0789 - val_mean_squared_error: 0.0111 - val_root_mean_squared_error: 0.1055
wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.
End epoch 0 of training; got log keys: ['loss', 'mean_absolute_error', 'mean_squared_error', 'root_mean_squared_error', 'val_loss', 'val_mean_absolute_error', 'val_mean_squared_e

20210527-153624/train/global_step,7
_timestamp,1622144232.68938
global_step,360
_step,32
_runtime,58
epoch,7
loss,0.0103
mean_absolute_error,0.07478
mean_squared_error,0.0103
root_mean_squared_error,0.10148
val_loss,0.00875


20210527-153624/train/global_step,▁▃▁▂▃▄▅▆▇█
_timestamp,▁▁▁█▁████████████████████████████
global_step,▁▁▂▃▃▄▄▅▅▅▅▆▆▆▇██
_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
_runtime,▁▂▂▂▂▃▃▄▄▅▅▆▆▇▇█
epoch,▁▂▃▄▅▆▇█
loss,█▂▂▁▁▁▁▁
mean_absolute_error,█▃▂▂▁▁▁▁
mean_squared_error,█▂▂▁▁▁▁▁
root_mean_squared_error,█▂▂▁▁▁▁▁
val_loss,█▆▃▂▂▁▁▁


wandb: WARNING When using several event log directories, please call wandb.tensorboard.patch(root_logdir="...") before wandb.init
wandb: WARNING When using several event log directories, please call wandb.tensorboard.patch(root_logdir="...") before wandb.init
Epoch 1/30
45/45 [==============================] - 1s 11ms/step - loss: 0.0359 - mean_absolute_error: 0.1404 - mean_squared_error: 0.0359 - root_mean_squared_error: 0.1894 - val_loss: 0.0131 - val_mean_absolute_error: 0.0879 - val_mean_squared_error: 0.0131 - val_root_mean_squared_error: 0.1145
wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.
End epoch 0 of training; got log keys: ['loss',

20210527-153735/train/global_step,7
_timestamp,1622144300.73762
global_step,360
_step,32
_runtime,53
epoch,7
loss,0.01391
mean_absolute_error,0.0883
mean_squared_error,0.01391
root_mean_squared_error,0.11792
val_loss,0.01101


...

20210527-154334/train/global_step,▁▂▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇██
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
loss,█▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
mean_absolute_error,█▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
mean_squared_error,█▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁
val_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: WARNING When using several event log directories, please call wandb.tensorboard.patch(root_logdir="...") before wandb.init
wandb: WARNING When using several event log directories, please call wandb.tensorboard.patch(root_logdir="...") before wandb.init
Epoch 1/30
44/44 [==============================] - 5s 56ms/step - loss: 0.7194 - mean_absolute_error: 0.6475 - mean_squared_error: 0.7194 - root_mean_squared_error: 0.8482 - val_loss: 0.0583 - val_mean_absolute_error: 0.1955 - val_mean_squared_error: 0.0583 - val_root_mean_squared_error: 0.2416
wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.
End epoch 0 of training; got log keys: ['loss',

20210527-154608/train/global_step,11
_timestamp,1622144840.81963
global_step,528
_step,48
_runtime,82
epoch,11
loss,0.05133
mean_absolute_error,0.16276
mean_squared_error,0.05133
root_mean_squared_error,0.22657
val_loss,0.01049


20210527-154608/train/global_step,▁▂▁▂▂▃▄▄▅▅▆▇▇█
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁██████
global_step,▁▁▂▂▂▃▃▃▃▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇█
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇█
epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▄▃▃▂▂▂▁▁▁▁▁
mean_absolute_error,█▅▄▃▃▂▂▂▂▁▁▁
mean_squared_error,█▄▃▃▂▂▂▁▁▁▁▁
root_mean_squared_error,█▅▄▃▃▃▂▂▂▁▁▁
val_loss,█▃▂▂▁▁▁▁▁▁▁▁


,loss,mean_absolute_error,mean_squared_error,root_mean_squared_error
scores,,,,
modelforecastdense:single_in_step,0.009,0.069,0.009,0.094
modelforecastdense:wide_in_step,0.011,0.081,0.011,0.105
modelforecastlstm:wide_in_step,0.013,0.090,0.013,0.113
modelforecastbilstm:wide_in_step,0.013,0.091,0.013,0.114
modelforecastlstmoneshot:wide_in_out,0.011,0.078,0.011,0.104
modelforecastlstmoneshot:wide_in_out_oneshot_hz_wk,0.010,0.074,0.010,0.099
modelforecastlstmoneshot:wide_in_out_oneshot_hz,0.010,0.079,0.010,0.102


In [25]:
# upload model artifacts  
tracker_cicd = lc.JobTrackInfo(**dict(
    name    = 'lineage',
    project = project_config['name'], 
    entity  = project_config['entity'],
    group   = ":".join(['exp', 'modeling']),
    # default main configuration used which will be added to 
    config  = config, 
    tags    = project_config['tags'] + ['artifact'],
    notes   = "service model experimentation model artifacts upload"
))
tracker_cicd.format_job_type(cycle=lc.JobTrackingCycle.Experiment, action=lc.ActionType.Upload)
with lc.LifeCycleSvc.execute(tracker_cicd) as cxt:
    artifact_svc = lc.LifeCycleArtifactSvc()
    for repo_name, uri_src, uri_dst in zip (
        ['model', 'figures'], 
        [config['experiment_parameters']['model_dir'], config['experiment_parameters']['figures_dir']], 
        ['training', 'figures']):
        # Artifact Add Config Files (./add_dir) to artifact
        artifact_config = lc.ArtifactInfo( **dict( 
            name        = f'{repo_name}_repo', 
            type        = lc.ArtifactStorage.Experiment, 
            description = f"model {repo_name} associated with training the dataset: {config['dataset']['id']}",
        ))
        artifact_svc.add(cxt, artifact_config, uri_src_path = uri_src,  uri_dst_path = uri_dst, 
            alias         = ['master', 'staging'], 
            write_request = lc.ArtifactWrite.AddDir
        )
    

wandb: Adding directory to artifact (./experiments/snapshots/m5-fcst-base/exports/artifacts/models)... Done. 0.1s
wandb: Adding directory to artifact (./experiments/snapshots/m5-fcst-base/exports/figures)... Done. 0.0s


In [38]:
with lc.LifeCycleSvc.execute(tracker_cicd) as cxt:
    import pandas 
    wandb.log({"scores":   wandb.Table(dataframe=pandas.DataFrame.from_dict(df_scores_eval.to_dict()))}, commit=True)
df_scores_eval

_runtime,9
_timestamp,1622146294
_step,0


_runtime,▁
_timestamp,▁
_step,▁


,loss,mean_absolute_error,mean_squared_error,root_mean_squared_error
scores,,,,
modelforecastdense:single_in_step,0.009,0.069,0.009,0.094
modelforecastdense:wide_in_step,0.011,0.081,0.011,0.105
modelforecastlstm:wide_in_step,0.013,0.090,0.013,0.113
modelforecastbilstm:wide_in_step,0.013,0.091,0.013,0.114
modelforecastlstmoneshot:wide_in_out,0.011,0.078,0.011,0.104
modelforecastlstmoneshot:wide_in_out_oneshot_hz_wk,0.010,0.074,0.010,0.099
modelforecastlstmoneshot:wide_in_out_oneshot_hz,0.010,0.079,0.010,0.102
